# MODEL CHARACTERIZATION

In this notebook we wil try to characterize the main feature of a TTN classifier depending on some of its parameters.
In particular we will compare a pure TTN classifier, with no activation regularization and batch normalization, with a fully optimzed one. After that the performance (accuracy) and time scaling in analyzed with different bond dimension, feature map and batch size.

This notebook is mainly composed of a series of for loop where models are trained with different hyperparameters. To see a complete explanation of each step of the training procedure please refer to the TTN_train_example notebook in this folder.

The results of this run are save in the JsonFile/Characterization/ folder and their results are plotted in the plot.ipynb notebook in the plots/ folder.

In [1]:
import os
import pandas as pd
import json
import time
import os
import math
import tensorflow        as tf
import tensornetwork     as tn
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import datetime

from tensorflow.keras.models      import Sequential
from tensorflow.keras             import regularizers

from sklearn.preprocessing        import MinMaxScaler
from sklearn.metrics              import roc_auc_score

In [3]:
import utils.preprocess as preprocess

from layers.TTN_SingleNode        import TTN_SingleNode
from ModelMaker                   import Make_SingleNode_Model

Using TensorFlow backend.


## DATA LOAD

Loading of the dataset and preprocessing

In [5]:
#dataset path
DATA_PATH = "../data/"
N = 1100000 #number of sample to load

#dataset loading
if os.path.isfile(DATA_PATH + "HIGGS.csv.gz"):
    data = pd.read_csv(
                DATA_PATH         + 'HIGGS.csv.gz'       ,  
                compression='gzip', error_bad_lines=False, 
                nrows=N           , header=None          
            )
elif os.path.isfile(DATA_PATH + "HIGGS.csv"):
    data = pd.read_csv(
                DATA_PATH + 'HIGGS.csv',    nrows=N  ,
                error_bad_lines=False  , header=None 
            )
else:
    print("Error: Data file not found")

In [6]:
#dataset preprocessing
x_train, x_val, x_test, y_train, y_val, y_test = \
    preprocess.Preprocess(
        data                       , 
        feature_map  = 'spherical' , #map typology
        map_order    = 2           , #map order
        con_order    = 2           , #number of contraction per site
        verbose      = True        , #verbose (print shapes for debugging)
        N_train      = 1000000     , #train set size
        N_val        = 50000       , #validation set size
        N_test       = 50000         #test set size
    )

Data shape
x_data shape:  (1100000, 28) y_data shape:  (1100000,)
Padded data shape
x_data shape:  (1100000, 32) y_data shape:  (1100000,)
Mapped data shape
x_data shape:  (1100000, 32, 3) y_data shape:  (1100000,)
Train, validation, test data shape
x_train shape:  (1000000, 32, 3) y_train shape:  (1000000,)
x_val   shape:  (50000, 32, 3) y_val   shape:  (50000,)
x_test  shape:  (50000, 32, 3) y_test  shape:  (50000,)


## "PURE" TTN vs OPTIMIZED ONE

Training of two model, the first one is a pure TTN without any sort of regularization and normalization, the other is a TTN with many standard ML optimization included.

In [14]:
#pure ttn model 
pure_ttn = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=15, activation=None, use_batch_norm=False,use_reg =False, n_contr=2)

In [15]:
#optimized ttn model
opti_ttn = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=10, activation='elu', use_batch_norm=True , n_contr=2)

In [16]:
pure_ttn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
opti_ttn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
#train pure model
pure_history = pure_ttn.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=150,  batch_size=5000 )

Train on 1000000 samples, validate on 50000 samples
Epoch 1/150
1000000/1000000 [==============================] - 9s 9us/sample - loss: 0.6843 - accuracy: 0.5559 - val_loss: 0.6722 - val_accuracy: 0.5820
Epoch 2/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6589 - accuracy: 0.6023 - val_loss: 0.6453 - val_accuracy: 0.6204
Epoch 3/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6402 - accuracy: 0.6295 - val_loss: 0.6331 - val_accuracy: 0.6386
Epoch 4/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6314 - accuracy: 0.6420 - val_loss: 0.6272 - val_accuracy: 0.6470
Epoch 5/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6263 - accuracy: 0.6508 - val_loss: 0.6270 - val_accuracy: 0.6511
Epoch 6/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6240 - accuracy: 0.6547 - val_loss: 0.6211 - val_accuracy: 0.6581
Epoch 7/150
1000000/1000000 [=

In [18]:
#save pure model results
with open("../JsonFiles/Characterizations/pure_model_history.json", 'w') as jf:
    json.dump({k:list(np.array(v).astype(float)) for k,v in pure_history.history.items()}, jf)

In [19]:
#train optimized model
opti_history = opti_ttn.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=150,  batch_size=5000 )

Train on 1000000 samples, validate on 50000 samples
Epoch 1/150
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.6885 - accuracy: 0.6008 - val_loss: 0.6347 - val_accuracy: 0.6583
Epoch 2/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.6131 - accuracy: 0.6779 - val_loss: 0.6014 - val_accuracy: 0.6817
Epoch 3/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.5893 - accuracy: 0.6980 - val_loss: 0.6083 - val_accuracy: 0.6828
Epoch 4/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.5777 - accuracy: 0.7064 - val_loss: 0.5710 - val_accuracy: 0.7109
Epoch 5/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.5670 - accuracy: 0.7138 - val_loss: 0.5634 - val_accuracy: 0.7156
Epoch 6/150
1000000/1000000 [==============================] - 6s 6us/sample - loss: 0.5573 - accuracy: 0.7202 - val_loss: 0.5551 - val_accuracy: 0.7209
Epoch 7/150
1000000/1000000 

In [20]:
#save optimized results
with open("../JsonFiles/Characterizations/opti_model_history.json", 'w') as jf:
    json.dump({k:list(np.array(v).astype(float)) for k,v in opti_history.history.items()}, jf)

## BOND DIMENSION 

Here we estimate the training time and model complexity dependence on the bond dimension. Due to the intensive computational task the performance evalutaion is not included in this notebook (jupyter is not able to manage the full the dataset). The performance evaluation can be done running the scripts/run_bond_dim.sh script to train several models and using the evaluate.py script to evaluate the performance of the trained models.

In [8]:
#time callback

class timecallback(tf.keras.callbacks.History):
    def __init__(self):
        self.times  = []
        self.epochs = []
        # use this value as reference to calculate cummulative time taken
        self.timetaken = tf.timestamp()
    def on_epoch_end(self,epoch,logs = {}):
        time_epoch = tf.timestamp()
        self.times.append(time_epoch - self.timetaken)
        self.timetaken = time_epoch
        self.epochs.append(epoch)
    def on_train_end(self,logs = {}):
        self.times = [t.numpy() for t in self.times]
        
timeCall = timecallback()

In [20]:
bond_params = {}


#loop over bond dimension 5 to 100 with step of 5
#save results of each run to a specific dict
for i in range(1,21):
    model = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=5*i, activation='elu', use_batch_norm=True , n_contr=2)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    temp_dict={}
    temp_dict['params'] = model.count_params()
    
    timeCall = timecallback()
    model.fit(x_train, y_train,validation_data=(x_val,y_val),
        epochs=10,  batch_size=5000, callbacks=[timeCall])
    temp_dict['mean_train_time'] = np.mean(np.array(timeCall.times))
    temp_dict['std_train_time'] = np.std(np.array(timeCall.times))
    start_time = time.time()
    model.predict(x_test)
    temp_dict['eval_time'] = time.time()-start_time
    bond_params['bd'+str(5*i)]= temp_dict
 

In [22]:
#save dictionary results
with open('../JsonFiles/Characterizations/bond_dim_behav.json', 'w') as jf:
    json.dump(bond_params, jf)

In [16]:
#study of the evaluation time
bond_params = {}

for i in range(1,21):
    model = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=5*i, activation='elu', use_batch_norm=True , n_contr=2)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    temp_dict={}
    ev_list = []
    for j in range(100):
        start_time = time.time()
        model.predict(x_test[0:1])
        ev_list.append(time.time()-start_time)
    temp_dict['mean_eval_time'] = np.mean(np.array(ev_list[1:]))
    temp_dict['std_eval_time'] = np.std (np.array(ev_list[1:]))
    bond_params['bd'+str(5*i)]= temp_dict
 

In [17]:
with open('../JsonFiles/Characterizations/bond_dim_behav_eval.json', 'w') as jf:
    json.dump(bond_params, jf)

## FEATURE MAP

Here we estimate the training and evaluation time and depending on the feature map. Due to the intensive computational task the performance evalutaion is not included in this notebook (jupyter is not able to manage all the dataset). The performance evaluation can be done running the scripts/run_maps.sh file.

In [36]:
sphe_dict = {}

for i in range(2,6):
    temp_dict={}
    x_train, x_val, x_test, y_train, y_val, y_test = preprocess.Preprocess(
        data, feature_map='spherical', map_order=i,  con_order=2, verbose=False, N_train= 1000000, N_val  =50000,N_test =50000)
    
    model = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=35, activation='elu', use_batch_norm=True , n_contr=2)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    timeCall = timecallback()
    model.fit(x_train, y_train,validation_data=(x_val,y_val),
        epochs=10,  batch_size=5000, callbacks=[timeCall])
    temp_dict['mean_train_time'] = np.mean(np.array(timeCall.times))
    temp_dict['std_train_time'] = np.std(np.array(timeCall.times))
    start_time = time.time()
    model.predict(x_test)
    temp_dict['eval_time'] = time.time()-start_time
    sphe_dict['order_'+str(i)]= temp_dict

Train on 1000000 samples, validate on 50000 samples
Epoch 1/10
1000000/1000000 [==============================] - 16s 16us/sample - loss: 0.6733 - accuracy: 0.6528 - val_loss: 0.6135 - val_accuracy: 0.6996
Epoch 2/10
1000000/1000000 [==============================] - 12s 12us/sample - loss: 0.5884 - accuracy: 0.7124 - val_loss: 0.5800 - val_accuracy: 0.7121
Epoch 3/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5610 - accuracy: 0.7264 - val_loss: 0.5793 - val_accuracy: 0.7036
Epoch 4/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5453 - accuracy: 0.7345 - val_loss: 0.5524 - val_accuracy: 0.7253
Epoch 5/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5355 - accuracy: 0.7396 - val_loss: 0.5343 - val_accuracy: 0.7395
Epoch 6/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5285 - accuracy: 0.7434 - val_loss: 0.5338 - val_accuracy: 0.7383
Epoch 7/10
1000000/10000

In [37]:
with open('../JsonFiles/Characterizations/map_spher_times.json', 'w') as jf:
    json.dump(sphe_dict, jf)

In [38]:
pol_dict = {}

for i in range(2,6):
    temp_dict={}
    x_train, x_val, x_test, y_train, y_val, y_test = preprocess.Preprocess(
        data, feature_map='polynomial', map_order=i,  con_order=2, verbose=False, N_train= 1000000, N_val  =50000,N_test =50000)
    
    model = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=35, activation='elu', use_batch_norm=True , n_contr=2)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    timeCall = timecallback()
    model.fit(x_train, y_train,validation_data=(x_val,y_val),
        epochs=10,  batch_size=5000, callbacks=[timeCall])
    temp_dict['mean_train_time'] = np.mean(np.array(timeCall.times))
    temp_dict['std_train_time'] = np.std(np.array(timeCall.times))
    start_time = time.time()
    model.predict(x_test)
    temp_dict['eval_time'] = time.time()-start_time
    pol_dict['order_'+str(i)]= temp_dict

Train on 1000000 samples, validate on 50000 samples
Epoch 1/10
1000000/1000000 [==============================] - 15s 15us/sample - loss: 0.6893 - accuracy: 0.6425 - val_loss: 0.6864 - val_accuracy: 0.6336
Epoch 2/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5963 - accuracy: 0.7091 - val_loss: 0.6175 - val_accuracy: 0.6760
Epoch 3/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5636 - accuracy: 0.7266 - val_loss: 0.5542 - val_accuracy: 0.7282
Epoch 4/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5453 - accuracy: 0.7354 - val_loss: 0.5469 - val_accuracy: 0.7341
Epoch 5/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5354 - accuracy: 0.7403 - val_loss: 0.5655 - val_accuracy: 0.7172
Epoch 6/10
1000000/1000000 [==============================] - 11s 11us/sample - loss: 0.5282 - accuracy: 0.7438 - val_loss: 0.5294 - val_accuracy: 0.7414
Epoch 7/10
1000000/10000

In [39]:
with open('../JsonFiles/Characterizations/map_poli_times.json', 'w') as jf:
    json.dump(pol_dict, jf)

## BATCH SIZE


Here we estimate the training and evaluation time and depending on the batch size.

In [7]:
batch_dict = {}

bs = np.logspace(1, 4, num=30, endpoint=True, dtype=np.int32, base = 10)
bs

array([   10,    12,    16,    20,    25,    32,    41,    52,    67,
          85,   108,   137,   174,   221,   280,   356,   452,   573,
         727,   923,  1172,  1487,  1887,  2395,  3039,  3856,  4893,
        6210,  7880, 10000], dtype=int32)

In [9]:
for b in bs:
    x_train, x_val, x_test, y_train, y_val, y_test = preprocess.Preprocess(data, feature_map='spherical', map_order=3,
                                                                       con_order=2, verbose=True, N_train= 1000000, N_val  =50000,N_test =50000)
    temp_dict={}
    model = Make_SingleNode_Model( input_shape=(x_train.shape[1:]),bond_dim=35, activation='elu', use_batch_norm=True , n_contr=2)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    timeCall = timecallback()
    model.fit(x_train[:100000], y_train[:100000],validation_data=(x_val,y_val),
        epochs=5,  batch_size=b, callbacks=[timeCall])
    temp_dict['mean_train_time'] = np.mean(np.array(timeCall.times))
    temp_dict['std_train_time'] = np.std(np.array(timeCall.times))
    start_time = time.time()
    batch_dict['bs_'+str(b)]= temp_dict

Data shape
x_data shape:  (1100000, 28) y_data shape:  (1100000,)
Padded data shape
x_data shape:  (1100000, 32) y_data shape:  (1100000,)
Mapped data shape
x_data shape:  (1100000, 32, 3) y_data shape:  (1100000,)
Train, validation, test data shape
x_train shape:  (1000000, 32, 3) y_train shape:  (1000000,)
x_val   shape:  (50000, 32, 3) y_val   shape:  (50000,)
x_test  shape:  (50000, 32, 3) y_test  shape:  (50000,)
Train on 100000 samples, validate on 50000 samples
Epoch 1/5
100000/100000 [==============================] - 115s 1ms/sample - loss: 0.6831 - accuracy: 0.6101 - val_loss: 0.6868 - val_accuracy: 0.5868
Epoch 2/5
100000/100000 [==============================] - 112s 1ms/sample - loss: 0.6525 - accuracy: 0.6333 - val_loss: 0.6395 - val_accuracy: 0.6560
Epoch 3/5
100000/100000 [==============================] - 112s 1ms/sample - loss: 0.6448 - accuracy: 0.6392 - val_loss: 0.6425 - val_accuracy: 0.6360
Epoch 4/5
100000/100000 [==============================] - 112s 1ms/sample

In [10]:
with open('../JsonFiles/Characterizations/batch_size_times.json', 'w') as jf:
    json.dump(batch_dict, jf)